In [1]:
sc

<SparkContext master=yarn appName=PySparkShell>

In [2]:
pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install sklearn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


# Importing the Libraries

In [4]:
import pyspark
import sklearn

from pyspark import SparkConf,SparkContext
from pyspark.sql.functions import regexp_extract
from pyspark.sql.types import *
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType
from pyspark.sql import Window
from pyspark.sql import DataFrameStatFunctions as statFunc


import pandas as pd
import sklearn.metrics as metrics
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import correlation
from sklearn.metrics.pairwise import pairwise_distances
import ipywidgets as widgets
from IPython.display import display, clear_output
from contextlib import contextmanager
import warnings
warnings.filterwarnings('ignore')

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

## Creating a spark session

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('MOVIE RECOMMENDATION BY COLLABORATIVE FILTERING USING THE NETFLIX DATA').getOrCreate()

## Creating the three dataframes for training, testing and movie data sets

In [6]:
from pyspark.sql.types import *

movies_df = StructType(
  [StructField('movieId', IntegerType()),
   StructField('yearOfRelease', IntegerType()),
   StructField('title', StringType())]
)
Training_df = StructType(
  [StructField('movieId', IntegerType()),
   StructField('userId', IntegerType()),
   StructField('ratings',DoubleType())]
)

Testing_df = StructType(
  [StructField('movieId', IntegerType()),
   StructField('userId', IntegerType()),
   StructField('ratings',DoubleType())]
)

In [7]:
#Creating the training,testing and movie dataframes

df_training = sqlContext.read.format('txt').options(inferSchema=True).schema(Training_df).csv('s3://dsci-6007-finalbucket/TrainingRatings.txt')
 
df_testing = sqlContext.read.format('txt').options(inferSchema=True).schema(Testing_df).csv('s3://dsci-6007-finalbucket/TestingRatings.txt')
 
df_movies = sqlContext.read.format('txt').options(inferSchema=True).schema(movies_df).csv('s3://dsci-6007-finalbucket/movie_titles.txt')
 

In [8]:
#Count of each dataframes    

training_count = df_training.count()
testing_count = df_testing.count()
movies_count = df_movies.count()

In [9]:
#Printing the data and count
print('There are %s samples in training set , %s samples in testing set and %s samples in movies in the datasets' % (training_count,testing_count, movies_count))
print('Training samples:')
df_training.show(5)
print('Testing samples:')
df_testing.show(5)
print('Movies samples:')
df_movies.show(5,truncate=False)

There are 3255352 samples in training set , 100478 samples in testing set and 17770 samples in movies in the datasets
Training samples:
+-------+-------+-------+
|movieId| userId|ratings|
+-------+-------+-------+
|      8|1744889|    1.0|
|      8|1395430|    2.0|
|      8|1205593|    4.0|
|      8|1488844|    4.0|
|      8|1447354|    1.0|
+-------+-------+-------+
only showing top 5 rows

Testing samples:
+-------+-------+-------+
|movieId| userId|ratings|
+-------+-------+-------+
|      8| 573364|    1.0|
|      8|2149668|    3.0|
|      8|1089184|    3.0|
|      8|2465894|    3.0|
|      8| 534508|    1.0|
+-------+-------+-------+
only showing top 5 rows

Movies samples:
+-------+-------------+----------------------------+
|movieId|yearOfRelease|title                       |
+-------+-------------+----------------------------+
|1      |2003         |Dinosaur Planet             |
|2      |2004         |Isle of Man TT 2004 Review  |
|3      |1997         |Character                

## HOW MANY DISTINCT USERS AND DISTINCT ITEMS ARE THERE IN THE TEST SET ?

In [10]:
#finding out the distinct users and items/movies in the testing set

print("The distinct items in testing set are :",df_testing.select('movieID').distinct().count())
print("The distinct users in testing set are :",df_testing.select('userID').distinct().count())


[Stage 12:>                                                         (0 + 1) / 1]21/12/15 16:49:01 WARN Utils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.


The distinct items in testing set are : 1701


[Stage 18:>                                                         (0 + 1) / 1]

The distinct users in testing set are : 27555


In [11]:
#finding out the distinct users and items(movies) in the training set 

print("The distinct items in training set are :",df_training.select('movieID').distinct().count())
print("The distinct users in training set are :",df_training.select('userID').distinct().count())

The distinct items in training set are : 1821


[Stage 30:===========================================>              (3 + 1) / 4]

The distinct users in training set are : 28978


# Finding Overall Rates for Users and Movies in both the sets 

## Some Initial analysis on training and testing sets 

In [12]:
#lets convert to a Pandas Dataframe 

training_pd = df_training.toPandas()

testing_pd = df_testing.toPandas()


In [13]:
#describing the training dataset 

training_pd.describe()

,movieId,userId,ratings
count,3.255352e+06,3.255352e+06,3.255352e+06
mean,8.724660e+03,1.327058e+06,3.481188e+00
std,5.107402e+03,7.626887e+05,1.082873e+00
min,8.000000e+00,7.000000e+00,1.000000e+00
25%,3.893000e+03,6.716970e+05,3.000000e+00
50%,8.825000e+03,1.322467e+06,4.000000e+00
75%,1.332600e+04,1.988873e+06,4.000000e+00
max,1.774200e+04,2.649285e+06,5.000000e+00


In [14]:
#describing the testing dataset
testing_pd.describe()

,movieId,userId,ratings
count,100478.000000,1.004780e+05,100478.000000
mean,8701.547792,1.329956e+06,3.479458
std,5098.075495,7.625041e+05,1.085280
min,8.000000,7.000000e+00,1.000000
25%,3893.000000,6.774300e+05,3.000000
50%,8699.000000,1.325031e+06,4.000000
75%,13298.000000,1.995052e+06,4.000000
max,17742.000000,2.649285e+06,5.000000


In [15]:
training_pd.tail()

,movieId,userId,ratings
3255347,17742,46222,3.0
3255348,17742,2534701,1.0
3255349,17742,208724,3.0
3255350,17742,483107,2.0
3255351,17742,1181331,2.0


In [16]:
testing_pd.tail()

,movieId,userId,ratings
100473,17742,1898310,2.0
100474,17742,716096,4.0
100475,17742,38115,3.0
100476,17742,2646347,5.0
100477,17742,273576,2.0


## Testing for null values in both the sets

In [17]:
training_pd.isnull().sum()

movieId    0
userId     0
ratings    0
dtype: int64

In [18]:
testing_pd.isnull().sum()

movieId    0
userId     0
ratings    0
dtype: int64

In [19]:
print("The counts of both the dataframes are ","Training dataset ==> ",len(training_pd),"Testing dataset ==>",len(testing_pd))

The counts of both the dataframes are  Training dataset ==>  3255352 Testing dataset ==> 100478


# Average Movie Ratings

In [20]:
#checking overall average rate of the movies with their counts  in the training set

averageMovieRatingsTraining = pd.DataFrame(training_pd.groupby('movieId')['ratings'].mean())

averageMovieRatingsTraining['counts'] =pd.DataFrame(training_pd.groupby('movieId')['ratings'].count())

averageMovieRatingsTraining.head()

,ratings,counts
movieId,,
8,3.055104,2831
28,3.760127,12244
43,2.310345,58
48,3.620648,1666
61,2.385965,57


In [21]:
#checking overall average rate of the movies with their counts  in the testing set

averageMovieRatingsTesting = pd.DataFrame(testing_pd.groupby('movieId')['ratings'].mean())

averageMovieRatingsTesting['counts'] =pd.DataFrame(testing_pd.groupby('movieId')['ratings'].count())

averageMovieRatingsTesting.head()

,ratings,counts
movieId,,
8,3.072917,96
28,3.698667,375
43,4.000000,1
48,3.622642,53
61,4.000000,3


In [22]:
averageMovieRatingsTesting.sort_values(by=['counts'],ascending=False).head(10)

,ratings,counts
movieId,,
6971,4.123305,811
4640,4.006614,756
6287,3.716418,737
9728,3.807365,706
8915,4.007194,695
4432,3.674855,692
8596,4.091172,691
6408,3.876471,680
1406,3.233129,652


# Average User Ratings

In [23]:
#checking overall average rate of the user with their counts  in the training set

averageUserRatingsTraining = pd.DataFrame(training_pd.groupby('userId')['ratings'].mean())

averageUserRatingsTraining['counts'] =pd.DataFrame(training_pd.groupby('userId')['ratings'].count())

averageUserRatingsTraining.head()

,ratings,counts
userId,,
7,3.903846,104
79,3.630952,84
199,3.943662,71
481,4.351351,74
769,3.193878,98


In [24]:
#checking overall average rate of the user with their counts  in the testing set

averageUserRatingsTesting = pd.DataFrame(testing_pd.groupby('userId')['ratings'].mean())

averageUserRatingsTesting['counts'] =pd.DataFrame(testing_pd.groupby('userId')['ratings'].count())

averageUserRatingsTesting.tail()

,ratings,counts
userId,,
2648869,2.777778,9
2648885,4.000000,5
2649120,5.000000,2
2649267,4.142857,7
2649285,2.333333,3


In [25]:
averageUserRatingsTesting.sort_values(by=['counts'],ascending=False).head(10)

,ratings,counts
userId,,
1664010,4.300000,70
2439493,1.307692,52
305344,1.711538,52
387418,1.941176,51
1314869,3.263158,38
2118461,4.117647,34
1932594,2.321429,28
491531,2.629630,27
2606799,2.888889,27


## ESTIMATED AVERAGE OVERLAP OF ITEMS FOR USERS

In [36]:
#Pivoting the table with userID as index and the movieId as columns for our training data

userItemRating = training_pd.pivot_table(index='userId', columns='movieId', values='ratings')

userItemRating.head()

movieId,8,28,43,48,61,64,66,92,96,111,...,17654,17660,17689,17693,17706,17725,17728,17734,17741,17742
userId,,,,,,,,,,,,,,,,,,,,,
7,5.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
481,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
769,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
testing_pd.head()

,movieId,userId,ratings
0,8,573364,1.0
1,8,2149668,3.0
2,8,1089184,3.0
3,8,2465894,3.0
4,8,534508,1.0


In [38]:
userItemRating.fillna(0, inplace = True)

userMovieRating = userItemRating.astype(np.int32)

userMovieRating.head()

movieId,8,28,43,48,61,64,66,92,96,111,...,17654,17660,17689,17693,17706,17725,17728,17734,17741,17742
userId,,,,,,,,,,,,,,,,,,,,,
7,5,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
79,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
199,0,0,0,0,0,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,0
481,0,0,0,0,0,0,0,0,0,5,...,0,0,0,0,0,0,0,0,0,0
769,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0



## I am using the top 10 users and movies which are having the most counts from the test dataset.


In [39]:
userMovieRating[17742].head(50)

userId
7       0
79      0
199     0
481     0
769     0
906     0
1310    0
1333    0
1427    0
1442    0
1457    0
1500    0
1527    0
1918    0
2000    0
2128    0
2213    0
2225    0
2307    0
2455    0
2469    0
2678    0
2787    0
2905    0
2976    0
3039    0
3321    0
3363    0
3458    0
3595    0
3604    0
3694    0
3718    0
3798    0
3870    0
3998    0
4057    0
4247    0
4306    0
4315    0
4409    0
4421    0
4576    0
4597    0
4679    0
4706    0
4783    0
4905    0
4906    0
4983    0
Name: 17742, dtype: int32

In [40]:
def average(user):
    summed =0
    count =0

    for i in range(len(user)):
        if(user[i]==0):
            summed = summed +0
        else:
            summed= user[i] +summed
            count= 1+ count
    avgg=summed/count
    return avgg

In [42]:
testing_pd.tail()

,movieId,userId,ratings
100473,17742,1898310,2.0
100474,17742,716096,4.0
100475,17742,38115,3.0
100476,17742,2646347,5.0
100477,17742,273576,2.0


In [43]:
user1664010= list(userMovieRating.loc[1664010])
avg1664010= average(user1664010)
print(avg1664010)

4.2384364820846905


In [44]:
user2439493 = list(userMovieRating.loc[2439493])

avg2439493= average(user2439493)

print(avg2439493)

1.225609756097561


In [45]:
user305344 = list(userMovieRating.loc[305344])

avg305344 = average(user305344)

print(avg305344)

1.904382470119522


In [46]:
user387418 = list(userMovieRating.loc[387418])

avg387418= average(user387418)

print(avg387418)

1.8405963302752293


In [47]:
user1314869 = list(userMovieRating.loc[1314869])

avg1314869= average(user1314869)

print(avg1314869)

2.970984455958549


In [48]:
user2118461 = list(userMovieRating.loc[2118461])

avg2118461= average(user2118461)

print(avg2118461)

4.088453747467927


In [49]:
user1932594 = list(userMovieRating.loc[1932594])

avg1932594 = average(user1932594)

print(avg1932594)

2.3195754716981134


In [50]:
user491531 = list(userMovieRating.loc[491531])

avg491531= average(user491531)

print(avg491531)

2.9102384291725105


In [51]:
user2606799 = list(userMovieRating.loc[2606799])

avg2606799 = average(user2606799)

print(avg2606799)

2.817982456140351


In [52]:
user727242 = list(userMovieRating.loc[727242])

avg727242 = average(user727242)

print(avg727242)

1.1890694239290989


In [53]:
ESTIMATED_AVERAGE_ITEMS_USERS = avg1314869+avg387418+avg305344+avg2439493+avg1664010+avg727242+avg2606799+avg491531+avg1932594+avg2118461

print("ESTIMATED AVERAGE OVERLAP OF ITEMS FOR USERS is : ",ESTIMATED_AVERAGE_ITEMS_USERS/10)

ESTIMATED AVERAGE OVERLAP OF ITEMS FOR USERS is :  2.550532902294355


### ESTIMATED AVERAGE OVERLAP OF ITEMS FOR USERS is :  2.550532902294355

## ESTIMATED AVERAGE OVERLAP OF USERS FOR ITEMS# 

In [56]:
#creating a utility matrix with movieId as index and the userId as columns to training data

movieUserRating = training_pd.pivot_table(index='movieId', columns='userId', values='ratings')

movieUserRating.head()

userId,7,79,199,481,769,906,1310,1333,1427,1442,...,2648572,2648589,2648730,2648734,2648853,2648869,2648885,2649120,2649267,2649285
movieId,,,,,,,,,,,,,,,,,,,,,
8,5.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,4.0,NaN,NaN,NaN,NaN,3.0,3.0,2.0,NaN,4.0,...,NaN,3.0,4.0,NaN,2.0,NaN,4.0,NaN,NaN,4.0
43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
movieUserRating.fillna(0, inplace = True)

movieUserRating = movieUserRating.astype(np.int32)

movieUserRating.head()

userId,7,79,199,481,769,906,1310,1333,1427,1442,...,2648572,2648589,2648730,2648734,2648853,2648869,2648885,2649120,2649267,2649285
movieId,,,,,,,,,,,,,,,,,,,,,
8,5,0,0,0,0,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
28,4,0,0,0,0,3,3,2,0,4,...,0,3,4,0,2,0,4,0,0,4
43,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
48,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
61,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
movie6971= list(movieUserRating.loc[6971])

avg6971= average(movie6971)

print(avg6971)

4.071815611748076


In [59]:

movie4640= list(movieUserRating.loc[4640])

avg4640 = average(movie4640)

print(avg4640)

4.047438894792774


In [60]:

movie6287= list(movieUserRating.loc[6287])

avg6287= average(movie6287)

print(avg6287)

3.7261099495756977


In [61]:

movie9728  = list(movieUserRating.loc[9728])

avg9728= average(movie9728)

print(avg9728)

3.8246204278812974


In [62]:
8915
movie8915 = list(movieUserRating.loc[8915])

avg8915 = average(movie8915)

print(avg8915)

3.9663373050469826


In [63]:
#4432
movie4432= list(movieUserRating.loc[4432])

avg4432 = average(movie4432)

print(avg4432)

3.6721471305118545


In [64]:
#8596
movie8596= list(movieUserRating.loc[8596])

avg8596 = average(movie8596)

print(avg8596)

4.103368832862421


In [65]:
#6408
movie6408= list(movieUserRating.loc[6408])

avg6408 = average(movie6408)

print(avg6408)

3.8013020096235492


In [66]:

movie1406= list(movieUserRating.loc[1406])

avg1406 = average(movie1406)

print(avg1406)

3.2807910426057876


In [67]:
movie1744= list(movieUserRating.loc[1744])

avg1744 = average(movie1744)

print(avg1744)

3.802730883813307


In [68]:

ESTIMATED_AVERAGE_USER_ITEMS = avg6971+avg4640+avg6287+avg9728+avg8915+avg1744+avg1406+avg6408+avg8596+avg4432

print("ESTIMATED AVERAGE OVERLAP OF USERS FOR ITEMS : ",ESTIMATED_AVERAGE_USER_ITEMS/10)

ESTIMATED AVERAGE OVERLAP OF USERS FOR ITEMS :  3.8296662088461746


### ESTIMATED AVERAGE OVERLAP OF USERS FOR ITEMS :  3.8296662088461746

# ALS Model Approach

In [69]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

In [70]:
#Joining the actual dataframes with movie dataframe for accurate understandings

df_training_join = df_training.join(df_movies,on=['movieId'],how='inner')

df_testing_join = df_testing.join(df_movies,on=['movieId'],how='inner')

### Approach 1, maxIter = 5 , regParam = 0.08

In [71]:
als = ALS(maxIter=5, regParam=0.08, userCol="userId", itemCol="movieId", ratingCol="ratings",
          coldStartStrategy="drop")
model1 = als.fit(df_training_join)



# Evaluate the model by computing the RMSE on the test data
predictions1 = model1.transform(df_testing_join)
evaluator = RegressionEvaluator(metricName="mse", labelCol="ratings",
                                predictionCol="prediction")
                                            
evaluator1 = RegressionEvaluator(metricName="rmse", labelCol="ratings",
                                predictionCol="prediction")



rmse = evaluator1.evaluate(predictions1)
mse = evaluator.evaluate(predictions1)

print("Mean squared error = " + str(mse))
print("Root-mean-square error = " + str(rmse))

[Stage 116:============================================>        (169 + 8) / 200]

Mean squared error = 0.7396199569226314
Root-mean-square error = 0.8600116027837251


### With the parameters maxIter = 5 , regParam = 0.08, Root-mean-square error is 0.860 and Mean squared error is 0.739

In [72]:
print("Following are the predictions results for the approach 1 model. \n")

predictions1.show()

Following are the predictions results for the approach 1 model. 

+-------+-------+-------+-------------+--------------------+----------+
|movieId| userId|ratings|yearOfRelease|               title|prediction|
+-------+-------+-------+-------------+--------------------+----------+
|      8| 573364|    1.0|         2004|What the #$*! Do ...| 3.0685322|
|      8|2149668|    3.0|         2004|What the #$*! Do ...| 2.9306412|
|      8|1089184|    3.0|         2004|What the #$*! Do ...| 2.2153425|
|      8|2465894|    3.0|         2004|What the #$*! Do ...| 2.3224208|
|      8| 534508|    1.0|         2004|What the #$*! Do ...| 2.9611382|
|      8| 992921|    4.0|         2004|What the #$*! Do ...| 2.6497087|
|      8| 595054|    4.0|         2004|What the #$*! Do ...| 2.8212795|
|      8|1298304|    4.0|         2004|What the #$*! Do ...| 3.4586916|
|      8|1661600|    4.0|         2004|What the #$*! Do ...| 3.1216502|
|      8| 553787|    2.0|         2004|What the #$*! Do ...| 2.8211422

In [73]:
userRecs = model1.recommendForAllUsers(10)

movieRecs = model1.recommendForAllItems(10)

In [74]:
#Recommendations for all users from model1 approach

pd_userRecs = userRecs.toPandas()

pd_userRecs.head()

,userId,recommendations
0,481,"[(4238, 5.191840648651123), (3033, 5.139437198..."
1,2678,"[(7569, 4.145359039306641), (3033, 4.109075069..."
2,3595,"[(12293, 4.385792255401611), (3290, 4.23582267..."
3,6460,"[(8933, 4.587021350860596), (12232, 4.55645847..."
4,7284,"[(8933, 5.285822868347168), (6522, 5.017945289..."


In [75]:
#Movie Recommendations for all users from model1 approach

pd_movieRecs = movieRecs.toPandas()

pd_movieRecs.head()

,movieId,recommendations
0,4190,"[(1502298, 4.437478542327881), (479924, 4.2170..."
1,3220,"[(447759, 5.086691856384277), (428653, 5.07802..."
2,11240,"[(1396680, 5.078673839569092), (428653, 4.9635..."
3,6110,"[(2239984, 4.9491047859191895), (428653, 4.937..."
4,8260,"[(2217027, 4.025911331176758), (17433, 3.95634..."


### Approach 2, maxIter = 10 , regParam = 0.01

In [76]:
als = ALS(maxIter=10, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="ratings",
          coldStartStrategy="drop")
model2 = als.fit(df_training_join)



# Evaluate the model by computing the RMSE on the test data
predictions2 = model2.transform(df_testing_join)
evaluator = RegressionEvaluator(metricName="mse", labelCol="ratings",
                                predictionCol="prediction")
                                            
evaluator1 = RegressionEvaluator(metricName="rmse", labelCol="ratings",
                                predictionCol="prediction")



rmse = evaluator1.evaluate(predictions2)
mse = evaluator.evaluate(predictions2)

print("Mean squared error = " + str(mse))
print("Root-mean-square error = " + str(rmse))

[Stage 346:=====================================>               (140 + 8) / 200]

Mean squared error = 0.707801248481052
Root-mean-square error = 0.8413092466394576


### With the parameters maxIter = 10 , regParam = 0.01, Root-mean-square error is 0.84 and Mean squared error is 0.70

In [77]:
predictions2.show()

+-------+-------+-------+-------------+--------------------+----------+
|movieId| userId|ratings|yearOfRelease|               title|prediction|
+-------+-------+-------+-------------+--------------------+----------+
|      8| 573364|    1.0|         2004|What the #$*! Do ...|  3.603221|
|      8|2149668|    3.0|         2004|What the #$*! Do ...| 3.4268858|
|      8|1089184|    3.0|         2004|What the #$*! Do ...| 2.9583192|
|      8|2465894|    3.0|         2004|What the #$*! Do ...| 2.0605848|
|      8| 534508|    1.0|         2004|What the #$*! Do ...| 3.3053515|
|      8| 992921|    4.0|         2004|What the #$*! Do ...| 2.5609016|
|      8| 595054|    4.0|         2004|What the #$*! Do ...| 2.9920435|
|      8|1298304|    4.0|         2004|What the #$*! Do ...| 3.3336272|
|      8|1661600|    4.0|         2004|What the #$*! Do ...|  3.533973|
|      8| 553787|    2.0|         2004|What the #$*! Do ...| 2.8933387|
|      8|1309839|    3.0|         2004|What the #$*! Do ...|   2

In [78]:
userRecs2 = model2.recommendForAllUsers(10)

movieRecs2 = model2.recommendForAllItems(10)

In [79]:
userRecs2.show()

[Stage 451:==================================================>   (93 + 7) / 100]

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|   481|[[6991, 8.501338]...|
|  2678|[[12952, 5.499856...|
|  3595|[[13218, 5.915258...|
|  6460|[[14089, 6.690787...|
|  7284|[[13590, 7.343672...|
|  7576|[[15567, 5.819330...|
|  9597|[[13218, 4.753746...|
| 15191|[[9854, 5.2554083...|
| 15846|[[6991, 6.6622877...|
| 20461|[[8657, 7.066132]...|
| 20774|[[7806, 4.630245]...|
| 26258|[[1951, 4.778234]...|
| 27608|[[6991, 7.448291]...|
| 28346|[[218, 5.2201567]...|
| 30941|[[3329, 5.866767]...|
| 30976|[[15480, 6.807095...|
| 31203|[[12952, 7.428676...|
| 36822|[[6287, 6.334367]...|
| 40851|[[6991, 5.506755]...|
| 41068|[[9854, 5.357501]...|
+------+--------------------+
only showing top 20 rows



In [80]:
movieRecs2.show()

[Stage 502:============================================>         (82 + 8) / 100]

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|   4190|[[2302897, 7.1306...|
|   3220|[[1596531, 5.8305...|
|  11240|[[1987007, 8.1333...|
|   6110|[[1874390, 5.3423...|
|   8260|[[1987007, 6.7983...|
|  16232|[[2567407, 5.1629...|
|   9492|[[868600, 8.20525...|
|    192|[[1358544, 6.7887...|
|   9482|[[2049584, 6.1950...|
|   6522|[[1658158, 8.0861...|
|  10082|[[1360119, 5.1963...|
|    122|[[1663569, 4.9832...|
|  12184|[[36822, 5.925970...|
|   9324|[[1256990, 5.7352...|
|   8354|[[973133, 6.18460...|
|   2366|[[27061, 5.272476...|
|  12046|[[973133, 6.84727...|
|  15216|[[2088784, 5.7237...|
|  17196|[[794999, 5.33111...|
|  14496|[[389739, 6.44611...|
+-------+--------------------+
only showing top 20 rows



### From the above two model performances we can say the RMSE value is comparitively less when maxIter = 10 and regParam = 0.01. Hence this would be our best ALS model.

In [98]:
from pyspark.sql import Row
my_user_id = 0

# Note that the movie IDs are the last number on each line. A common error was to use the number of ratings as the movie ID.
my_rated_movies = [
          (240,0,5),
     (290,1,5),
     (890,2,5),
     (100,3,5),
     (100,4,5),
     (523,5,5),
     (219,6,5),
     (229,7,5),
     (209,8,5),
     (320,9,2)
     # The format of each line is (my_user_id, movie ID, your rating)
     # For example, to give the movie "Star Wars: Episode IV - A New Hope (1977)" a five rating, you would add the following line:
     #   (my_user_id, 260, 5),
]

my_ratings_df = sqlContext.createDataFrame(my_rated_movies, ['movieId','userId','rating'])
print ('My movie ratings:')
display(my_ratings_df.limit(10))
my_ratings_df.show(10, truncate=False)

My movie ratings:


DataFrame[movieId: bigint, userId: bigint, rating: bigint]

+-------+------+------+
|movieId|userId|rating|
+-------+------+------+
|240    |0     |5     |
|290    |1     |5     |
|890    |2     |5     |
|100    |3     |5     |
|100    |4     |5     |
|523    |5     |5     |
|219    |6     |5     |
|229    |7     |5     |
|209    |8     |5     |
|320    |9     |2     |
+-------+------+------+

